In [58]:
import itertools
import logging
import os
import pickle

import gensim
from gensim.utils import smart_open, simple_preprocess
import numpy as np
from cltk.stop.greek.stops import STOPS_LIST
STOPS_LIST = [simple_preprocess(stop, deacc=True)[0] for stop in STOPS_LIST if len(simple_preprocess(stop, deacc=True)) > 0]

logging.basicConfig(format='%(levelname)s : %(message)s', level=logging.INFO)
logging.root.level = logging.INFO  # ipython sometimes messes up the logging setup; restore

# Load data

In [9]:
# Load pre-saved BoW
# Save BoW
user_dir = os.path.expanduser('~/cltk_data/user_data/')
try:
    os.makedirs(user_dir)
except FileExistsError:
    pass
bow_path = os.path.join(user_dir, 'bow_lda_gensim.mm')

mm_corpus = gensim.corpora.MmCorpus(bow_path)
print(mm_corpus)

MmCorpus(1484 documents, 84120 features, 3961081 non-zero entries)


In [10]:
print(next(iter(mm_corpus)))

[(0, 1.0), (1, 1.0), (2, 1.0), (3, 1.0), (4, 1.0), (5, 1.0), (6, 1.0), (7, 2.0), (8, 1.0), (9, 1.0), (10, 1.0), (11, 1.0), (12, 1.0), (13, 1.0), (14, 1.0), (15, 1.0), (16, 1.0), (17, 1.0), (18, 1.0), (19, 1.0), (20, 1.0), (21, 1.0), (22, 1.0), (23, 1.0), (24, 1.0), (25, 1.0), (26, 1.0), (27, 1.0), (28, 1.0), (29, 1.0), (30, 1.0), (31, 1.0), (32, 1.0), (33, 1.0), (34, 1.0), (35, 1.0), (36, 1.0), (37, 1.0), (38, 1.0), (39, 1.0), (40, 1.0), (41, 1.0), (42, 1.0), (43, 1.0), (44, 1.0), (45, 1.0), (46, 1.0), (47, 1.0), (48, 2.0), (49, 1.0), (50, 1.0), (51, 1.0), (52, 1.0), (53, 1.0), (54, 1.0), (55, 1.0), (56, 1.0), (57, 1.0), (58, 1.0), (59, 1.0), (60, 1.0), (61, 1.0), (62, 1.0), (63, 1.0), (64, 1.0), (65, 1.0), (66, 1.0), (67, 1.0), (68, 1.0), (69, 1.0)]


# Semantic transformations

http://radimrehurek.com/topic_modeling_tutorial/2%20-%20Topic%20Modeling.html#Semantic-transformations

## LDA

In [15]:
# Save for reuse
with open(os.path.expanduser('~/cltk_data/user_data/tlg_bow_id2word.dict'), 'rb') as file_open:
    id2word_tlg = pickle.load(file_open)

In [16]:
# Quick testing using just a part of the corpus

# use fewer documents during training, LDA is slow
clipped_corpus = gensim.utils.ClippedCorpus(mm_corpus, 100)
%time lda_model = gensim.models.LdaModel(clipped_corpus,
                                         num_topics=10,
                                         id2word=id2word_tlg,
                                         passes=4)

CPU times: user 4min 16s, sys: 54.4 s, total: 5min 10s
Wall time: 4min 7s


In [18]:
lda_model.print_topics(-1)  # print a few most important words for each LDA topic

[(0,
  '0.001*"διαφωνει" + 0.001*"διακεκαυμενη" + 0.000*"εναλλαγαι" + 0.000*"θερινη" + 0.000*"βουλομεθα" + 0.000*"οικετων" + 0.000*"καιροι" + 0.000*"λυσαωνθειηδεση" + 0.000*"ωργισμενους" + 0.000*"δεομενω"'),
 (1,
  '0.002*"νενευκασιν" + 0.001*"αδωμος" + 0.001*"μυουμενους" + 0.001*"νοτιωτερα" + 0.001*"διακειμενων" + 0.001*"αιστωτηριον" + 0.001*"προειρημενον" + 0.001*"επιταξ" + 0.001*"δηλουντος" + 0.001*"ασπετον"'),
 (2,
  '0.001*"αγνοιαν" + 0.001*"ακουσας" + 0.001*"ακρατον" + 0.001*"κινδυνευοντι" + 0.001*"λαμπτηρα" + 0.001*"εγενετο" + 0.000*"στοιχους" + 0.000*"ερωτος" + 0.000*"λαβομενη" + 0.000*"εναγισαντες"'),
 (3,
  '0.000*"ωργισμενους" + 0.000*"διαβοσκεσθαι" + 0.000*"απολυοντος" + 0.000*"ετοιμως" + 0.000*"σαρδονυχες" + 0.000*"νεα" + 0.000*"συμβαινει" + 0.000*"διαφωνει" + 0.000*"εξηρτημενων" + 0.000*"πενεσται"'),
 (4,
  '0.000*"νημερτεστατε" + 0.000*"φιλολογος" + 0.000*"απλανεσιν" + 0.000*"δηλαδη" + 0.000*"επιπλοκης" + 0.000*"θρασυλλωι" + 0.000*"διαβοσκεσθαι" + 0.000*"επωνομαζετο" + 0

## TI-IDF + LSI

In [25]:
# first train tfidf model
# this modifies the feature weights of each word
%time tfidf_model = gensim.models.TfidfModel(mm_corpus, id2word=id2word_tlg)

INFO : collecting document frequencies
INFO : PROGRESS: processing document #0
INFO : calculating IDF weights for 1484 documents and 84119 features (3961081 matrix non-zeros)


CPU times: user 6.34 s, sys: 12 ms, total: 6.35 s
Wall time: 6.29 s


In [26]:
# then run lsi, which reduces dimensionality
%time lsi_model = gensim.models.LsiModel(tfidf_model[mm_corpus], id2word=id2word_tlg, num_topics=200)

INFO : using serial LSI version on this node
INFO : updating model with new documents
INFO : preparing a new chunk of documents
INFO : using 100 extra samples and 2 power iterations
INFO : 1st phase: constructing (1161925, 300) action matrix
INFO : orthonormalizing (1161925, 300) action matrix
INFO : 2nd phase: running dense svd on (300, 1484) matrix
INFO : computing the final decomposition
INFO : keeping 200 factors (discarding 19.480% of energy spectrum)
INFO : processed documents up to #1484
INFO : topic #0(5.232): 0.097*"καταπελταισι" + 0.096*"νοτιωτερα" + 0.093*"ηνεχυρασμενον" + 0.091*"διακειμενων" + 0.077*"αδωμος" + 0.075*"απωλειαν" + 0.071*"βεβαιωσον" + 0.066*"ροιζησαν" + 0.064*"οικειον" + 0.058*"λυσαωνθειηδεση"
INFO : topic #1(3.674): -0.138*"καταπελταισι" + -0.135*"νοτιωτερα" + -0.130*"διακειμενων" + -0.126*"ηνεχυρασμενον" + -0.106*"απωλειαν" + -0.092*"ροιζησαν" + 0.089*"μουνον" + -0.081*"βεβαιωσον" + -0.081*"οικειον" + -0.072*"νοτιωτεραν"
INFO : topic #2(3.159): 0.174*"νεφους

CPU times: user 4min 2s, sys: 1min 4s, total: 5min 7s
Wall time: 58.3 s


In [27]:
# for the first doc of the TLG corpus, here are the LSI scores for each of the 200 topics
print(next(iter(lsi_model[tfidf_model[mm_corpus]])))

[(0, 0.024289037065511311), (1, 0.014077893150839215), (2, -0.019146078135534279), (3, -0.018767966018989272), (4, -0.00099884333481120103), (5, -0.0051913135174345487), (6, -0.0014277098382469583), (7, -0.018818824919966906), (8, 0.0056374952350573453), (9, -0.0023221873083177956), (10, -0.0081668241786406002), (11, -0.019996384848007791), (12, 0.014338300486609062), (13, -0.00010303379864312642), (14, 0.0021088994788161335), (15, 0.0098757069667030732), (16, 0.0090732013832107734), (17, 0.0081570602885538118), (18, -0.0098302970910077361), (19, 0.01134303047809858), (20, -0.0042868477183203814), (21, -0.0019021128579354079), (22, 0.0083599609888375184), (23, -0.0024645892529614947), (24, -0.0089570567476525975), (25, 0.013241931401700954), (26, -0.0079806310637484165), (27, 0.0037053228065251193), (28, 0.002579960226988357), (29, 0.016675254728906144), (30, 0.001139717059664551), (31, 0.017415051948121849), (32, 0.02155588300179399), (33, -0.014399643194274744), (34, 0.00864178216383

## Save corpora

In [30]:
# cache the transformed corpora to disk, for use in later notebooks
path_lda = os.path.join(user_dir, 'gensim_tlg_lda.mm')
path_tfidf = os.path.join(user_dir, 'gensim_tlg_tfidf.mm')
path_lsi= os.path.join(user_dir, 'gensim_tlg_lsa.mm')
%time gensim.corpora.MmCorpus.serialize(path_lda, lda_model[mm_corpus])
%time gensim.corpora.MmCorpus.serialize(path_tfidf, tfidf_model[mm_corpus])
%time gensim.corpora.MmCorpus.serialize(path_lsi, lsi_model[tfidf_model[mm_corpus]])

INFO : storing corpus in Matrix Market format to /home/kyle/cltk_data/user_data/gensim_tlg_lda.mm
INFO : saving sparse matrix to /home/kyle/cltk_data/user_data/gensim_tlg_lda.mm
INFO : PROGRESS: saving document #0
INFO : PROGRESS: saving document #1000
INFO : saved 1484x10 matrix, density=56.617% (8402/14840)
INFO : saving MmCorpus index to /home/kyle/cltk_data/user_data/gensim_tlg_lda.mm.index
INFO : storing corpus in Matrix Market format to /home/kyle/cltk_data/user_data/gensim_tlg_tfidf.mm
INFO : saving sparse matrix to /home/kyle/cltk_data/user_data/gensim_tlg_tfidf.mm
INFO : PROGRESS: saving document #0


CPU times: user 24.2 s, sys: 1min 11s, total: 1min 35s
Wall time: 12.6 s


INFO : PROGRESS: saving document #1000
INFO : saved 1484x84120 matrix, density=3.173% (3961081/124834080)
INFO : saving MmCorpus index to /home/kyle/cltk_data/user_data/gensim_tlg_tfidf.mm.index
INFO : storing corpus in Matrix Market format to /home/kyle/cltk_data/user_data/gensim_tlg_lsa.mm
INFO : saving sparse matrix to /home/kyle/cltk_data/user_data/gensim_tlg_lsa.mm


CPU times: user 14 s, sys: 364 ms, total: 14.4 s
Wall time: 14 s


INFO : PROGRESS: saving document #0
INFO : PROGRESS: saving document #1000
INFO : saved 1484x200 matrix, density=100.000% (296800/296800)
INFO : saving MmCorpus index to /home/kyle/cltk_data/user_data/gensim_tlg_lsa.mm.index


CPU times: user 9.81 s, sys: 112 ms, total: 9.92 s
Wall time: 9.9 s


## Transform "unseen" documents

What is the best way to get LDA scores on a corpus used for training a model?

In [42]:
# LDA

def tokenize(text):
    # https://radimrehurek.com/gensim/utils.html#gensim.utils.simple_preprocess
    tokens = [token for token in simple_preprocess(text, deacc=True)]
    return [token for token in tokens if token not in STOPS_LIST]

doc = "ἐπειδὴ πᾶσαν πόλιν ὁρῶμεν κοινωνίαν τινὰ οὖσαν καὶ πᾶσαν κοινωνίαν ἀγαθοῦ τινος ἕνεκεν συνεστηκυῖαν （τοῦ γὰρ εἶναι δοκοῦντος ἀγαθοῦ χάριν πάντα πράττουσι πάντες）, δῆλον ὡς πᾶσαι μὲν ἀγαθοῦ τινος στοχάζονται, μάλιστα δὲ [5] καὶ τοῦ κυριωτάτου πάντων ἡ πασῶν κυριωτάτη καὶ πάσας περιέχουσα τὰς ἄλλας. αὕτη δ᾽ ἐστὶν ἡ καλουμένη πόλις καὶ ἡ κοινωνία ἡ πολιτική. ὅσοι μὲν οὖν οἴονται πολιτικὸν καὶ βασιλικὸν καὶ οἰκονομικὸν καὶ δεσποτικὸν εἶναι τὸν αὐτὸν οὐ καλῶς λέγουσιν （πλήθει γὰρ καὶ ὀλιγότητι νομίζουσι [10] διαφέρειν ἀλλ᾽ οὐκ εἴδει τούτων ἕκαστον, οἷον ἂν μὲν ὀλίγων, δεσπότην, ἂν δὲ πλειόνων, οἰκονόμον, ἂν δ᾽ ἔτι πλειόνων, πολιτικὸν ἢ βασιλικόν, ὡς οὐδὲν διαφέρουσαν μεγάλην οἰκίαν ἢ μικρὰν πόλιν: καὶ πολιτικὸν δὲ καὶ βασιλικόν, ὅταν μὲν αὐτὸς ἐφεστήκῃ, βασιλικόν, ὅταν [15] δὲ κατὰ τοὺς λόγους τῆς ἐπιστήμης τῆς τοιαύτης κατὰ μέρος ἄρχων καὶ ἀρχόμενος, πολιτικόν: ταῦτα δ᾽ οὐκ ἔστιν ἀληθῆ）: δῆλον δ᾽ ἔσται τὸ λεγόμενον ἐπισκοποῦσι κατὰ τὴν ὑφηγημένην μέθοδον. ὥσπερ γὰρ ἐν τοῖς ἄλλοις τὸ σύνθετον μέχρι τῶν ἀσυνθέτων ἀνάγκη διαιρεῖν （ταῦτα γὰρ ἐλάχιστα [20] μόρια τοῦ παντός）, οὕτω καὶ πόλιν ἐξ ὧν σύγκειται σκοποῦντες ὀψόμεθα καὶ περὶ τούτων μᾶλλον, τί τε διαφέρουσιν ἀλλήλων καὶ εἴ τι τεχνικὸν ἐνδέχεται λαβεῖν περὶ ἕκαστον τῶν ῥηθέντων."
doc = ' '.join(simple_preprocess(doc))

In [43]:
# transform text into the bag-of-words space
bow_vector = id2word_tlg.doc2bow(tokenize(doc))
print([(id2word_tlg[id], count) for id, count in bow_vector])

[('αυτη', 1), ('τινα', 1), ('τοις', 1), ('ουσαν', 1), ('τινος', 2), ('ουδεν', 1), ('τας', 1), ('πολιν', 3), ('παντες', 1), ('παντα', 1), ('αλλας', 1), ('ειναι', 2), ('πασαν', 2), ('παντων', 1), ('λεγουσιν', 1), ('αυτον', 1), ('ων', 1), ('μερος', 1), ('ουτω', 1), ('οιον', 1), ('εστιν', 2), ('δηλον', 2), ('λαβειν', 1), ('ταυτα', 2), ('διαφερουσαν', 1), ('λογους', 1), ('ωσπερ', 1), ('πολις', 1), ('τουτων', 2), ('ρηθεντων', 1), ('οταν', 2), ('μαλλον', 1), ('λεγομενον', 1), ('επειδη', 1), ('αληθη', 1), ('πραττουσι', 1), ('χαριν', 1), ('οικιαν', 1), ('μαλιστα', 1), ('αλλοις', 1), ('ενεκεν', 1), ('καλουμενη', 1), ('μεγαλην', 1), ('εκαστον', 2), ('αρχων', 1), ('αλληλων', 1), ('μεχρι', 1), ('πληθει', 1), ('οιονται', 1), ('παντος', 1), ('συνθετον', 1), ('αρχομενος', 1), ('κοινωνιαν', 2), ('τοιαυτης', 1), ('καλως', 1), ('πλειονων', 2), ('νομιζουσι', 1), ('αναγκη', 1), ('πασας', 1), ('περιεχουσα', 1), ('δεσποτην', 1), ('πασαι', 1), ('οσοι', 1), ('μεθοδον', 1), ('ειδει', 1), ('επιστημης', 1), ('πασ

In [45]:
# transform into LDA space
lda_vector = lda_model[bow_vector]
print(lda_vector)

[(2, 0.62834060088914145), (4, 0.075956794886348669), (5, 0.07265613621925443), (7, 0.036576776832934622), (9, 0.18233695136950517)]
0.001*"αγνοιαν" + 0.001*"ακουσας" + 0.001*"ακρατον" + 0.001*"κινδυνευοντι" + 0.001*"λαμπτηρα" + 0.001*"εγενετο" + 0.000*"στοιχους" + 0.000*"ερωτος" + 0.000*"λαβομενη" + 0.000*"εναγισαντες"


In [46]:
# print the document's single most prominent LDA topic
print(lda_model.print_topic(max(lda_vector, key=lambda item: item[1])[0]))

0.001*"αγνοιαν" + 0.001*"ακουσας" + 0.001*"ακρατον" + 0.001*"κινδυνευοντι" + 0.001*"λαμπτηρα" + 0.001*"εγενετο" + 0.000*"στοιχους" + 0.000*"ερωτος" + 0.000*"λαβομενη" + 0.000*"εναγισαντες"


In [47]:
# transform into LSI space
lsi_vector = lsi_model[tfidf_model[bow_vector]]
print(lsi_vector)

[(0, 0.024566044502988167), (1, 0.015083771441513003), (2, -0.0053303162617581688), (3, -0.0049469294214897746), (4, -0.004304577822397548), (5, -0.0019232322736995517), (6, 0.0012844223351456006), (7, -0.014361304541287518), (8, 0.0029628230116745817), (9, -0.001991805827179696), (10, -0.0038961163189403162), (11, -0.0017520317514794631), (12, -0.003289379651920948), (13, 0.013984478154812633), (14, -0.0042467976096142246), (15, -0.001134755589540278), (16, -3.7748879427212047e-05), (17, 0.0026908339487799688), (18, 0.0008648525754869072), (19, 0.0041513971222094598), (20, -0.0089440015914239723), (21, -0.0039957917322863508), (22, -0.0039182934453001663), (23, -0.0016527724004994603), (24, -0.00052822506831432989), (25, 0.0080844104724639085), (26, -0.0062247602598571213), (27, -0.0044917328067377452), (28, -0.0037489807849336193), (29, 0.00082110267265825731), (30, -0.00062982415302895579), (31, 0.0012224945919592724), (32, 0.00034178180065024514), (33, -0.0035051594171728109), (34,

In [48]:
# print the document's single most prominent LSI topic (not interpretable like LDA!)
print(lsi_model.print_topic(max(lsi_vector, key=lambda item: abs(item[1]))[0]))

0.140*"περιαυχενιον" + -0.082*"λεγοιεν" + 0.072*"απονοηθεντων" + -0.065*"κριθαι" + -0.064*"κλαπεντας" + 0.063*"ζωνας" + -0.062*"περιπαθης" + -0.061*"αμμωνος" + 0.061*"λυσικλει" + 0.061*"λαμβανοντα"


## Save models

In [49]:
path_lda = os.path.join(user_dir, 'gensim_tlg_lda.model')
path_tfidf = os.path.join(user_dir, 'gensim_tlg_tfidf.model')
path_lsi= os.path.join(user_dir, 'gensim_tlg_lsa.model')

# store all trained models to disk
lda_model.save(path_lda)
lsi_model.save(path_lsi)
tfidf_model.save(path_tfidf)

INFO : saving LdaState object under /home/kyle/cltk_data/user_data/gensim_tlg_lda.model.state, separately None
INFO : storing np array 'sstats' to /home/kyle/cltk_data/user_data/gensim_tlg_lda.model.state.sstats.npy
INFO : saved /home/kyle/cltk_data/user_data/gensim_tlg_lda.model.state
INFO : saving LdaModel object under /home/kyle/cltk_data/user_data/gensim_tlg_lda.model, separately ['expElogbeta', 'sstats']
INFO : storing np array 'expElogbeta' to /home/kyle/cltk_data/user_data/gensim_tlg_lda.model.expElogbeta.npy
INFO : not storing attribute dispatcher
INFO : not storing attribute id2word
INFO : not storing attribute state
INFO : saved /home/kyle/cltk_data/user_data/gensim_tlg_lda.model
INFO : saving Projection object under /home/kyle/cltk_data/user_data/gensim_tlg_lsa.model.projection, separately None
INFO : storing np array 'u' to /home/kyle/cltk_data/user_data/gensim_tlg_lsa.model.projection.u.npy
INFO : saved /home/kyle/cltk_data/user_data/gensim_tlg_lsa.model.projection
INFO : 

# Evaluation

Read insighful intro at http://radimrehurek.com/topic_modeling_tutorial/2%20-%20Topic%20Modeling.html#Evaluation

# Word intrusion

> For each trained topic, they take its first ten words, then substitute one of them with another, randomly chosen word (intruder!) and see whether a human can reliably tell which one it was. If so, the trained topic is topically coherent (good); if not, the topic has no discernible theme (bad)

In [54]:
# select top 50 words for each of the 20 LDA topics
top_words = [[word for word, _ in lda_model.show_topic(topicno, topn=50)] for topicno in range(lda_model.num_topics)]
print(top_words)

[['διαφωνει', 'διακεκαυμενη', 'εναλλαγαι', 'θερινη', 'βουλομεθα', 'οικετων', 'καιροι', 'λυσαωνθειηδεση', 'ωργισμενους', 'δεομενω', 'απολαβουσα', 'αφοβοι', 'διεσφαλται', 'αντεστραμμενην', 'πεντακοσιων', 'ευτελων', 'αποτυφλωσιν', 'συμμαρτυρει', 'νεκρωσιν', 'αγχονη', 'δυνατωτερου', 'αγορητης', 'διεγγυηθεισα', 'εκουσιως', 'θα', 'μελικερτης', 'εγγραφηναι', 'απολλω', 'πολλοστον', 'συγκριτικον', 'οριζοντα', 'υπειπον', 'ελκτικη', 'ψαλτριων', 'αξιοπιστω', 'διαβολην', 'επανελαβεν', 'πολυφημην', 'εξισασουσι', 'εριζει', 'πυθαγορειων', 'μεμνησθε', 'κεγχρεων', 'δηλουντος', 'ολιγως', 'βασεσιν', 'αυριον', 'μηθενος', 'αρκτου', 'μανθανοντας'], ['νενευκασιν', 'αδωμος', 'μυουμενους', 'νοτιωτερα', 'διακειμενων', 'αιστωτηριον', 'προειρημενον', 'επιταξ', 'δηλουντος', 'ασπετον', 'ζωδιακος', 'αντεστραμμενην', 'νοτιωτεραν', 'απωλειαν', 'ειρημενας', 'ενθεντι', 'ανετελλεν', 'αλλοφυλω', 'συνουσιαν', 'μεταλλα', 'αμυριος', 'αφορωσης', 'κωπηλατας', 'τριακοντ', 'ευφοριαν', 'προσδοκωση', 'πηγη', 'ιστορει', 'εδιδαξαμεν'

In [59]:
# get all top 50 words in all 20 topics, as one large set
all_words = set(itertools.chain.from_iterable(top_words))

print("Can you spot the misplaced word in each topic?")

# for each topic, replace a word at a different index, to make it more interesting
replace_index = np.random.randint(0, 10, lda_model.num_topics)

replacements = []
for topicno, words in enumerate(top_words):
    other_words = all_words.difference(words)
    replacement = np.random.choice(list(other_words))
    replacements.append((words[replace_index[topicno]], replacement))
    words[replace_index[topicno]] = replacement
    print("%i: %s" % (topicno, ' '.join(words[:10])))

Can you spot the misplaced word in each topic?
0: προκομισαντας διακεκαυμενη εναλλαγαι θερινη βουλομεθα οικετων καιροι λυσαωνθειηδεση ωργισμενους δεομενω
1: νενευκασιν αδωμος μυουμενους νοτιωτερα διακειμενων διακεκαυμενη προειρημενον επιταξ δηλουντος ασπετον
2: αγνοιαν ακουσας ακρατον νωχος λαμπτηρα εγενετο στοιχους ερωτος λαβομενη εναγισαντες
3: ωργισμενους διαβοσκεσθαι απολυοντος ετοιμως σαρδονυχες παθεσιν συμβαινει διαφωνει εξηρτημενων πενεσται
4: νημερτεστατε φιλολογος απλανεσιν δηλαδη επιπλοκης θρασυλλωι διαβοσκεσθαι μανθανοντας κρειτ φερωνυμον
5: ρυπος περιτρεπομενα καταπελταισι διαφωνει σικελιωτας διακειμενων αποκτενουσι αποπλυσεως καθιεσαν εχομεν
6: διαβοσκεσθαι δηλαδη παρακαθηται τυχης σχηματιζομενου εμφαινουσιν ανισχη ελληνικη κεκληκας υποκριτη
7: κριθαι συντεμνε καθαυανει αναστησας καθισταντο εκκλητευεσθαι βαδιζουσιν πολυευκτον ταφον αρασκη
8: σαμαιου λακεδαιμονα ωρμημενους εμπεδοκλεους ακρατον αναπεπεικως διηπορησθαι καθηστο βοηθησειαν βαινειν
9: διαφωνει οικετων θερινη δια

In [60]:
print("Actual replacements were:")
print(list(enumerate(replacements)))

Actual replacements were:
[(0, ('διαφωνει', 'προκομισαντας')), (1, ('αιστωτηριον', 'διακεκαυμενη')), (2, ('κινδυνευοντι', 'νωχος')), (3, ('νεα', 'παθεσιν')), (4, ('επωνομαζετο', 'μανθανοντας')), (5, ('νοτιωτερα', 'διαφωνει')), (6, ('πετευρον', 'ελληνικη')), (7, ('κραυαλλιδαι', 'αναστησας')), (8, ('αγομενης', 'ακρατον')), (9, ('καιροι', 'πυλωροι'))]


## Split doc

> We'll split each document into two parts, and check that 1) topics of the first half are similar to topics of the second 2) halves of different documents are mostly dissimilar

In [68]:
# this function first defined in pt. 1
def iter_tlg(tlg_dir):
    file_names = os.listdir(tlg_dir)
    for file_name in file_names:
        file_path = os.path.join(tlg_dir, file_name)
        with open(file_path) as file_open:
            file_read = file_open.read()
        tokens = tokenize(file_read)
        # ignore short docs
        if len(tokens) < 50:
            continue
        yield file_name, tokens

# evaluate on 1k documents **not** used in LDA training
tlg_preprocessed = os.path.expanduser('~/cltk_data/greek/text/tlg/plaintext/')
doc_stream = (tokens for _, tokens in iter_tlg(tlg_preprocessed))  # generator
test_docs = list(itertools.islice(doc_stream, 100, 200))  # ['πανυ', 'καλως', ...], [...], ...]

In [93]:
def intra_inter(model, test_docs, num_pairs=10000):
    # split each test document into two halves and compute topics for each half
    part1 = [model[id2word_tlg.doc2bow(tokens[: len(tokens) // 2])] for tokens in test_docs]
    part2 = [model[id2word_tlg.doc2bow(tokens[len(tokens) // 2 :])] for tokens in test_docs]
    
    # print computed similarities (uses cossim)
    print("average cosine similarity between corresponding parts (higher is better):")
    print(np.mean([gensim.matutils.cossim(p1, p2) for p1, p2 in zip(part1, part2)]))

    random_pairs = np.random.randint(0, len(test_docs), size=(num_pairs, 2))
    print("average cosine similarity between {} random parts (lower is better):".format(num_pairs))    
    print(np.mean([gensim.matutils.cossim(part1[i[0]], part2[i[1]]) for i in random_pairs]))

In [94]:
print("LDA results:")
intra_inter(lda_model, test_docs)

LDA results:
average cosine similarity between corresponding parts (higher is better):
0.979140673252
average cosine similarity between 10000 random parts (lower is better):
0.967707618176


In [95]:
print("LSI results:")
intra_inter(lsi_model, test_docs)

LSI results:
average cosine similarity between corresponding parts (higher is better):
0.759159191756
average cosine similarity between 10000 random parts (lower is better):
0.556656437511


# How to use on corpus

Just send convert list of tokens to bow, then send to model: `print(lda_model[id2word_tlg.doc2bow(tokens)])`

In [110]:
for title, tokens in iter_tlg(tlg_preprocessed):
    #print(title, tokens[:10])  # print the article title and its first ten tokens
    print(title)
    print(lda_model[id2word_tlg.doc2bow(tokens)])
    print('')


TLG2346.TXT
[(2, 0.77020063255871529), (4, 0.050162662423303819), (7, 0.17759554457537929)]

TLG1389.TXT
[(1, 0.077333330171680217), (2, 0.81018572424497037), (4, 0.029298653619360696), (7, 0.021474921514088502), (9, 0.050474519337949199)]

TLG0404.TXT
[(1, 0.02048927219191567), (2, 0.94907945485028755), (4, 0.029755487376673027)]

TLG0235.TXT
[(2, 0.9770261992926299), (4, 0.015441547453474994)]

TLG0535.TXT
[(1, 0.034813206783092389), (2, 0.71772563652669896), (3, 0.014290175511093661), (4, 0.090466392452987279), (5, 0.0652139733662397), (7, 0.018721921830686009), (9, 0.055341464745294021)]

TLG0507.TXT
[(1, 0.10259625305824292), (2, 0.41373140862637803), (4, 0.14912418250002712), (5, 0.12201859250883705), (7, 0.16176346421143439), (9, 0.050147784476549384)]

TLG1816.TXT
[(1, 0.024000415375272359), (2, 0.86327207953334895), (4, 0.028950050907302946), (5, 0.02272453752338216), (7, 0.035543720007879025), (9, 0.013945883813084248)]

TLG0476.TXT
[(1, 0.038183750715045969), (2, 0.927640218

[(1, 0.069888217070998571), (2, 0.64448915701583642), (4, 0.089020211952983888), (5, 0.042890057823670724), (6, 0.011129856079415947), (7, 0.011270471246184608), (9, 0.12631930064772245)]

TLG1981.TXT
[(1, 0.045691126629257443), (2, 0.58258777952129892), (4, 0.14980354368691123), (5, 0.053443544905473178), (6, 0.013665414301995341), (7, 0.011206202183390277), (9, 0.14122732778841451)]

TLG0536.TXT
[(1, 0.06322154078436043), (2, 0.63305787536916736), (4, 0.091796696406017286), (5, 0.041752574137819749), (6, 0.010607335927409351), (7, 0.022514360540822131), (9, 0.12916746112920538)]

TLG2892.TXT
[(1, 0.072246425111774648), (2, 0.60421214549573854), (4, 0.099253071342821977), (5, 0.042318682870410296), (6, 0.01113069321787143), (7, 0.013314380492660937), (9, 0.14904826039828561)]

TLG0708.TXT
[(1, 0.065907188974519595), (2, 0.59699961161037252), (4, 0.068045913347469303), (5, 0.062391041627117928), (7, 0.014126885586319825), (9, 0.17621434088462185)]

TLG0560.TXT
[(1, 0.07556054987674983)

TLG2053.TXT
[(1, 0.075322130943838703), (2, 0.57416739529996885), (4, 0.079105519699793861), (5, 0.12076588969837167), (7, 0.018327324544550362), (9, 0.11942905780024501)]

TLG2724.TXT
[(1, 0.066612060759416974), (2, 0.64550208869589509), (4, 0.09093727284058635), (5, 0.029332802477220511), (6, 0.012021685633571071), (7, 0.011136100141568907), (9, 0.1412276041289014)]

TLG2409.TXT
[(1, 0.091872512730068079), (2, 0.56785348476337216), (4, 0.1299139998037778), (5, 0.061409036023643081), (7, 0.013516054805162887), (9, 0.12319559413957686)]

TLG3047.TXT
[(1, 0.079553014250171167), (2, 0.59392037446965384), (4, 0.10982638598638292), (5, 0.043900471948448018), (7, 0.016207462016938727), (9, 0.13993572191847267)]

TLG2016.TXT
[(1, 0.088889005475262003), (2, 0.59856603324982582), (4, 0.080842742480600749), (5, 0.05782779229088654), (6, 0.014043343433780258), (7, 0.014009185171581534), (9, 0.13855264820115626)]

TLG0492.TXT
[(0, 0.21817828478628989), (1, 0.098637978497196327), (2, 0.38927102897

TLG0385.TXT
[(0, 0.011783909500877545), (1, 0.094537420973154526), (2, 0.56578916988255934), (4, 0.13664390544132948), (5, 0.053483113175003746), (6, 0.011900993497750027), (7, 0.014868179279539264), (9, 0.10272253131762678)]

TLG2233.TXT
[(1, 0.087460841809573406), (2, 0.6304066905054706), (4, 0.081859773592593596), (5, 0.038095833920206261), (9, 0.14426869280952115)]

TLG0022.TXT
[(0, 0.020878742329852568), (1, 0.066927606483027452), (2, 0.44280879235894194), (4, 0.1526767665840684), (5, 0.076975935032917844), (6, 0.043812223072308303), (7, 0.053093038987204653), (9, 0.1367715911057765)]

TLG4153.TXT
[(1, 0.091842361799974459), (2, 0.55949424311748308), (4, 0.11929343462492914), (5, 0.060113225152617661), (6, 0.013414500585467412), (7, 0.019228331060384955), (9, 0.12051721789336259)]

TLG0488.TXT
[(2, 0.8201098688626911), (4, 0.042496904249366911), (5, 0.082420878494971719), (9, 0.049363633289034745)]

TLG2031.TXT
[(1, 0.071003031247748219), (2, 0.60713013203693689), (4, 0.0874124457

TLG2702.TXT
[(1, 0.074124290873043813), (2, 0.59312994396837859), (4, 0.099812944286737287), (5, 0.043377626477459449), (6, 0.012754476217933941), (7, 0.01429182964013773), (9, 0.1527847171922124)]

TLG1139.TXT
[(1, 0.073226133939135449), (2, 0.59336044004681276), (4, 0.095858356184563967), (5, 0.05776580405459366), (6, 0.017698095929933357), (7, 0.025448469677961091), (9, 0.12442170994044105)]

TLG0517.TXT
[(2, 0.68490652663200779), (4, 0.054463269137564288), (7, 0.051727295894395349), (9, 0.19818633379606684)]

TLG3069.TXT
[(1, 0.11039553073632791), (2, 0.49625162715738441), (4, 0.15268845009546997), (5, 0.072184647618962927), (6, 0.01343123501523541), (7, 0.022443669517204775), (9, 0.11356906575345808)]

TLG0554.TXT
[(1, 0.08484572710824527), (2, 0.57500356252226126), (4, 0.11294186043169888), (5, 0.069448748660001933), (6, 0.010244220563705822), (7, 0.013487442720421888), (9, 0.12130340076716016)]

TLG3128.TXT
[(0, 0.010029053551449389), (1, 0.066802861521995077), (2, 0.54284556620

...

TLG2800.TXT
[(1, 0.089626288587359576), (2, 0.55143451412445332), (4, 0.12416130685899024), (5, 0.05904812711850238), (6, 0.010034078614709107), (7, 0.012188161328228345), (9, 0.14525348297754678)]

TLG4000.TXT
[(1, 0.082322304974555144), (2, 0.5651112959518968), (4, 0.098981806207071879), (5, 0.060573345423419855), (7, 0.013523102540927586), (9, 0.16046301781085892)]

TLG9021.TXT
[(1, 0.060808239981321725), (2, 0.63858371887095855), (4, 0.081861853422160902), (5, 0.03837692686405135), (7, 0.016045084273885901), (9, 0.14939204912371418)]

TLG3130.TXT
[(1, 0.10205693431557011), (2, 0.50456662450025536), (4, 0.15023742925893488), (5, 0.072487603654309313), (6, 0.020188723693626048), (7, 0.021697851852905934), (9, 0.1112679748875387)]

TLG1588.TXT
[(0, 0.021779381085833733), (1, 0.10152560310881353), (2, 0.43762449357168798), (4, 0.14981108719117084), (5, 0.052760938531821198), (6, 0.020975436870609623), (7, 0.055184046551904037), (9, 0.15957542105609621)]

TLG0502.TXT
[(1, 0.039471420308

[(1, 0.053135780450873207), (2, 0.67169222441252063), (4, 0.077074050563865529), (5, 0.070252092005890998), (7, 0.02018425174079426), (9, 0.1008239086570525)]

TLG2298.TXT
[(1, 0.13399062370687953), (2, 0.48534192992296321), (4, 0.17035042881382584), (9, 0.20317272192137156)]

TLG1801.TXT
[(1, 0.07429456549556411), (2, 0.51962134956832129), (4, 0.14578375710234584), (5, 0.071939195583001936), (6, 0.021276119553921068), (7, 0.030696553450857107), (9, 0.13530905376522884)]

TLG1737.TXT
[(2, 0.49883521309673146), (4, 0.3934949690773677), (5, 0.099131022266088623)]

TLG0540.TXT
[(1, 0.076731805735002448), (2, 0.65827369399848568), (4, 0.098931455732204132), (5, 0.043223887054170067), (7, 0.016823476227627598), (9, 0.089397217062593806)]

TLG2460.TXT
[(1, 0.32351123164574919), (2, 0.46500747305694679), (4, 0.16992638374625649), (5, 0.017019697697845265), (9, 0.020502164303066062)]

TLG4110.TXT
[(1, 0.068343070877419879), (2, 0.59537764639334878), (4, 0.10085577060343756), (5, 0.053560081318

TLG0748.TXT
[(1, 0.070811747849655729), (2, 0.56847992740322539), (4, 0.10359278568818188), (5, 0.059694978637492145), (6, 0.011361102048722167), (7, 0.024423491038550228), (9, 0.15185911553469994)]

TLG2012.TXT
[(0, 0.018936402941099967), (1, 0.067820510083906771), (2, 0.65115845847230036), (4, 0.15297936575553536), (5, 0.071465570037818421), (9, 0.016772018396264937)]

TLG1632.TXT
[(1, 0.074638132669363672), (2, 0.5414014685892089), (4, 0.12196103116907622), (5, 0.048613477603846836), (6, 0.011996227601220108), (7, 0.016998774310317939), (9, 0.16892844488852712)]

TLG1747.TXT
[(1, 0.046107476520064458), (2, 0.6560226449628962), (4, 0.13369262651416747), (5, 0.027146723425140878), (9, 0.12513671395909215)]

TLG0063.TXT
[(1, 0.078502683031564324), (2, 0.55438277409502812), (4, 0.12666502504429), (5, 0.028834055019344797), (6, 0.012397110154501786), (7, 0.03274838764048648), (9, 0.15912101755316163)]

TLG2034.TXT
[(1, 0.068273091180197712), (2, 0.62997149647449846), (4, 0.08710606088808

TLG7051.TXT
[(1, 0.05932988043034338), (2, 0.63516326980574322), (4, 0.063178161178252104), (5, 0.028884362465224522), (6, 0.012108927972110915), (7, 0.0102571794939931), (9, 0.18462881803275175)]

TLG4338.TXT
[(1, 0.087500793637327268), (2, 0.59928926595003573), (4, 0.10019781441453457), (5, 0.08137655186663989), (9, 0.12777161579021934)]

TLG2168.TXT
[(1, 0.066127963121050598), (2, 0.64664249260445594), (5, 0.049139050542971789), (9, 0.22791878550065403)]

TLG0380.TXT
[(1, 0.072975687879785853), (2, 0.58308844162238083), (4, 0.082359468612125453), (5, 0.077416451032025033), (6, 0.02472596919966729), (7, 0.052928094238478232), (8, 0.011389277737998903), (9, 0.09467597285362471)]

TLG0066.TXT
[(1, 0.070926759881074219), (2, 0.62233609085542696), (4, 0.10217816938306894), (5, 0.051874477453048795), (6, 0.012100012768809676), (7, 0.019524990447554239), (9, 0.11401660424980495)]

TLG0059.TXT
[(1, 0.075790106688360448), (2, 0.65948977646423912), (4, 0.090263533709317556), (5, 0.03485848828

TLG2023.TXT
[(1, 0.059913382847830786), (2, 0.62176741291080373), (4, 0.092582038946801531), (5, 0.040220682420336662), (6, 0.010833881689896536), (7, 0.015363510157864628), (9, 0.15330120814558604)]

TLG2506.TXT
[(1, 0.13023901175386599), (2, 0.37525000416894827), (4, 0.084523205047392622), (5, 0.35547150150978779), (9, 0.051429182400569991)]

TLG1127.TXT
[(1, 0.072944521435799284), (2, 0.56993100556749587), (4, 0.034782337426070599), (5, 0.03492663204818381), (6, 0.033670854086981464), (7, 0.020608134726592696), (9, 0.2330145871700316)]

TLG1736.TXT
[(1, 0.03044925772921752), (2, 0.586298357591303), (4, 0.1453162719668486), (7, 0.046253071192684514), (9, 0.1849243996256078)]

TLG3002.TXT
[(1, 0.062662232750699418), (2, 0.59669790829531633), (4, 0.072784786312179134), (5, 0.038745265938923953), (9, 0.21860384474020761)]

TLG4139.TXT
[(1, 0.074700155265828586), (2, 0.61731319376938032), (4, 0.10174765415046903), (5, 0.043470707346181151), (7, 0.010429989251275539), (9, 0.13576910670161

# Visualization

Following: http://nbviewer.jupyter.org/github/bmabey/pyLDAvis/blob/master/notebooks/pyLDAvis_overview.ipynb

In [112]:
lda_model.show_topics()

[(0,
  '0.001*"διαφωνει" + 0.001*"διακεκαυμενη" + 0.000*"εναλλαγαι" + 0.000*"θερινη" + 0.000*"βουλομεθα" + 0.000*"οικετων" + 0.000*"καιροι" + 0.000*"λυσαωνθειηδεση" + 0.000*"ωργισμενους" + 0.000*"δεομενω"'),
 (1,
  '0.002*"νενευκασιν" + 0.001*"αδωμος" + 0.001*"μυουμενους" + 0.001*"νοτιωτερα" + 0.001*"διακειμενων" + 0.001*"αιστωτηριον" + 0.001*"προειρημενον" + 0.001*"επιταξ" + 0.001*"δηλουντος" + 0.001*"ασπετον"'),
 (2,
  '0.001*"αγνοιαν" + 0.001*"ακουσας" + 0.001*"ακρατον" + 0.001*"κινδυνευοντι" + 0.001*"λαμπτηρα" + 0.001*"εγενετο" + 0.000*"στοιχους" + 0.000*"ερωτος" + 0.000*"λαβομενη" + 0.000*"εναγισαντες"'),
 (3,
  '0.000*"ωργισμενους" + 0.000*"διαβοσκεσθαι" + 0.000*"απολυοντος" + 0.000*"ετοιμως" + 0.000*"σαρδονυχες" + 0.000*"νεα" + 0.000*"συμβαινει" + 0.000*"διαφωνει" + 0.000*"εξηρτημενων" + 0.000*"πενεσται"'),
 (4,
  '0.000*"νημερτεστατε" + 0.000*"φιλολογος" + 0.000*"απλανεσιν" + 0.000*"δηλαδη" + 0.000*"επιπλοκης" + 0.000*"θρασυλλωι" + 0.000*"διαβοσκεσθαι" + 0.000*"επωνομαζετο" + 0

In [115]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()

In [116]:
pyLDAvis.gensim.prepare(lda_model, mm_corpus, id2word_tlg)

/home/kyle/cltk/venv_pypi/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  topic_term_dists = topic_term_dists.ix[topic_order]


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
2      35.631149        1       1  0.088158 -0.015850
5      18.019666        1       2  0.004897  0.004299
9      15.382394        1       3 -0.023505 -0.035567
1      14.775720        1       4 -0.019135 -0.060255
4       9.804799        1       5  0.014334  0.026662
6       2.534828        1       6 -0.006589  0.017878
7       1.471482        1       7 -0.011628  0.019876
8       1.387386        1       8 -0.015753  0.029870
0       0.900709        1       9 -0.016105 -0.011614
3       0.091868        1      10 -0.014673  0.024701, topic_info=      Category         Freq            Term        Total  loglift  logprob
term                                                                      
420    Default  2051.000000    διαβοσκεσθαι  2051.000000  30.0000  30.0000
11366  Default  3781.000000      νενευκασιν  3781.000000  29.0000  29.0000
62635  Default  1253.000000         σαμαιου  1253.000000  28.0000  28.0000
23907  Default  1227.000000     λακεδαιμονα  1227.000000  27.0000  27.0000
15168  Default  3914.000000        διαφωνει  3914.000000  26.0000  26.0000
20772  Default  2326.000000    διακεκαυμενη  2326.000000  25.0000  25.0000
43773  Default  2694.000000          αδωμος  2694.000000  24.0000  24.0000
20877  Default  2412.000000          θερινη  2412.000000  23.0000  23.0000
1525   Default  3117.000000         οικετων  3117.000000  22.0000  22.0000
62637  Default   725.000000      ωρμημενους   725.000000  21.0000  21.0000
20235  Default  1915.000000       εναλλαγαι  1915.000000  20.0000  20.0000
31015  Default  2109.000000      απολαβουσα  2109.000000  19.0000  19.0000
18892  Default  2615.000000          αφοβοι  2615.000000  18.0000  18.0000
12295  Default  1519.000000           μοισι  1519.000000  17.0000  17.0000
2010   Default  2441.000000      μυουμενους  2441.000000  16.0000  16.0000
18229  Default   618.000000    εμπεδοκλεους   618.000000  15.0000  15.0000
20209  Default  1697.000000          καιροι  1697.000000  14.0000  14.0000
15186  Default  2587.000000       νοτιωτερα  2587.000000  13.0000  13.0000
353    Default  1039.000000           τυχης  1039.000000  12.0000  12.0000
20241  Default  1498.000000       βουλομεθα  1498.000000  11.0000  11.0000
10887  Default  1560.000000     ωργισμενους  1560.000000  10.0000  10.0000
3729   Default   730.000000    διαμαρτυριας   730.000000   9.0000   9.0000
31011  Default  2083.000000      διεσφαλται  2083.000000   8.0000   8.0000
14105  Default  1890.000000           ρυπος  1890.000000   7.0000   7.0000
13390  Default  1841.000000  λυσαωνθειηδεση  1841.000000   6.0000   6.0000
95     Default  2850.000000         ακουσας  2850.000000   5.0000   5.0000
43340  Default  2363.000000     διακειμενων  2363.000000   4.0000   4.0000
24707  Default  1500.000000         δεομενω  1500.000000   3.0000   3.0000
15125  Default  2066.000000  αντεστραμμενην  2066.000000   2.0000   2.0000
38245  Default  1420.000000     αιστωτηριον  1420.000000   1.0000   1.0000
...        ...          ...             ...          ...      ...      ...
10973  Topic10     0.176177     αγνωμοσυνην    28.192110   1.9173 -11.0521
6656   Topic10     0.260041     επιτηδευειν    70.901078   1.3844 -10.6627
57089  Topic10     0.175087       λαθομενος    29.054394   1.8809 -11.0583
10884  Topic10     0.351857          οποιον   157.247417   0.8902 -10.3603
10887  Topic10     0.897035     ωργισμενους  1560.406341  -0.4688  -9.4245
55683  Topic10     0.297529          βηθηλη   105.035347   1.1260 -10.5280
10907  Topic10     0.433098         ετοιμως   368.476280   0.2464 -10.1526
46796  Topic10     0.425578      σαρδονυχες   416.511320   0.1064 -10.1701
420    Topic10     0.608432    διαβοσκεσθαι  2051.915438  -1.1308  -9.8127
55536  Topic10     0.381296     εξηρτημενων   362.820952   0.1345 -10.2800
2806   Topic10     0.267057        ενεκαλει    97.906524   1.0883 -10.6361
5093   To